In [ ]:
from nuscenes.nuscenes import NuScenes
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import os
import cv2
import PIL
import numpy as np
from celluloid import Camera
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import LidarPointCloud

mpl.rc('figure', max_open_warning = 0)
%matplotlib inline

In [ ]:
nusc = NuScenes(version='v1.0-mini', dataroot='./data/nusc', verbose=True)

# 1a)

In [ ]:
my_sample = nusc.sample[0]
sample_data = nusc.get('sample_data', my_sample['data']['LIDAR_TOP'])
pcl_path = sample_data['filename']
nusc.render_sample_data(sample_data['token'])
pc =LidarPointCloud.from_file(os.path.join('./data/nusc',pcl_path))
color = pc.points[3] / np.max(pc.points[3])
plt.scatter(pc.points[0], pc.points[1], c=color, alpha=0.2, s=0.3)

# plt.savefig("sample.jpg")

# image = cv2.imread("sample.jpg")
# # convert the image into RGB format
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# # reshape the image to a 2D array of pixels and 3 color values (RGB)
# pixel_values = image.reshape((-1, 3))

# # pixel_values = np.delete(pc.points, 2, 0).T
# # print(test.shape)
# # convert to float
# pixel_values = np.float32(pixel_values)

# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1)
# # number of clusters (K)
# k = 12
# _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
# # convert back to 8 bit values
# centers = np.uint8(centers)

# # flatten the labels array
# labels = labels.flatten()
# # convert all pixels to the color of the centroids
# segmented_image = centers[labels.flatten()]
# # reshape back to the original image dimension
# segmented_image = segmented_image.reshape(image.shape)
# # show the image
# plt.figure(figsize = (16,8))
# plt.imshow(segmented_image)

In [ ]:
X = pc.points[0:2].T
kmeans = KMeans(n_clusters=69)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=0.2, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.5);

# 1b)

In [ ]:
def get_lidar_pointcloud(sample_token):
    sample_record = nusc.get('sample', sample_token)
    pointsensor_channel = 'LIDAR_TOP'
    pointsensor_token = sample_record['data'][pointsensor_channel]
    pointsensor = nusc.get('sample_data', pointsensor_token)
    pcl_path = nusc.get('sample_data', pointsensor_token)['filename']
    pc =LidarPointCloud.from_file(os.path.join('./data/nusc',pcl_path))
    return pointsensor_token, pc.points
def rotate_pointcloud(cloud, quaternion):
    def rot(an_array):
        return quaternion.rotate(an_array)
    return np.array(list(map(rot, cloud)))
def visualize_absolute(ax, absolute_points, ego_pose, van_translation, relativeboxvectors):
    axes = plt.gca()
    return ax.scatter(absolute_points[0], absolute_points[1], c='red', s=0.3), ax.scatter(ego_pose['translation'][0], ego_pose['translation'][1]),ax.plot([van_translation[0] + relativeboxvector[0] for relativeboxvector in relativeboxvectors], [van_translation[1] + relativeboxvector[1] for relativeboxvector in relativeboxvectors])[0]
def create_bounding_box(annotation_token):
    '''
    creates bounding box used for plotting
    '''
    van_annotation = nusc.get('sample_annotation', annotation_token)
    relativeboxvectors = [(van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0)]
    return np.array(relativeboxvectors), van_annotation['translation']

In [ ]:
ann_tokens = my_sample['anns']
X_data = np.array([])
y_data = np.array([])
for i,current_van_annotation_token in enumerate(ann_tokens):
    van_annotation = nusc.get('sample_annotation', current_van_annotation_token)
    pointsensor_token, points = get_lidar_pointcloud(my_sample['token'])
    calibrated_sensor_token = nusc.get('sample_data', pointsensor_token)['calibrated_sensor_token']
    calibrated_sensor = nusc.get('calibrated_sensor', calibrated_sensor_token)
    ego_pose = nusc.get('ego_pose', nusc.get('sample_data', pointsensor_token)['ego_pose_token'])

    ### Rotate points around sensor and ego rotation
    pointsensor_quaternion = Quaternion(calibrated_sensor['rotation'])
    ego_quaternion = Quaternion(ego_pose['rotation'])

    sensor_rotated_points = np.dot(pointsensor_quaternion.rotation_matrix, points[:3,:])
    rotated_points = np.dot(ego_quaternion.rotation_matrix, sensor_rotated_points)

    # use broadcasting to add translation to x and y dimension
    absolute_points = rotated_points+np.array(ego_pose['translation'][:3]).reshape(-1,1)

    ### rotate points by inverse bounding box rotation (translation to its reference frame unnecessary for only filtering)
    reverse_van_quaternion = Quaternion(w=-1*van_annotation['rotation'][0], x=van_annotation['rotation'][1], y=van_annotation['rotation'][2], z=van_annotation['rotation'][3])
    points_bounding_rotation = np.dot(reverse_van_quaternion.rotation_matrix, absolute_points)

    #new stuff
    relativeboxvectors, position = create_bounding_box(current_van_annotation_token)

    rotated_box = relativeboxvectors+reverse_van_quaternion.rotate(position)

    x_max = rotated_box[:,0].max()
    x_min = rotated_box[:,0].min()

    y_max = rotated_box[:,1].max()
    y_min = rotated_box[:,1].min()

    ## Remove points that are outside the bounding box
    mask = np.ones(points_bounding_rotation.shape[1], dtype=bool)

    mask = np.logical_and(mask, points_bounding_rotation[0] > x_min)
    mask = np.logical_and(mask, points_bounding_rotation[0] < x_max)

    mask = np.logical_and(mask, points_bounding_rotation[1] > y_min)
    mask = np.logical_and(mask, points_bounding_rotation[1] < y_max)

    points_filtered_rotated = points_bounding_rotation[:,mask]

    ### rotate points back
    van_quaternion = Quaternion(van_annotation['rotation'])
    points_filtered_absolute = np.dot(van_quaternion.rotation_matrix, points_filtered_rotated)

    for px in points_filtered_absolute[0]:
        X_data = np.append(X_data, px)
    for py in points_filtered_absolute[1]:
        y_data = np.append(y_data, py)
X_data = np.reshape(X_data, (X_data.shape[0], 1))
y_data = np.reshape(y_data, (y_data.shape[0], 1))
data = np.append(X_data, y_data, axis=1)
# Implement K-means
kmeans = KMeans(n_clusters=69)
kmeans.fit(data)
y_kmeans = kmeans.predict(data)
plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=1, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.2);

# 1c)